In [ ]:
import ee
import time
import geepy
import math

ee.Initialize()

In [ ]:
config = geepy.params.configParams('input.json')

ls = ee.FeatureCollection(config.params['lsTiles'])
tiles = [geepy.feature.getTiles(ls, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [ ]:
tileNames = [tiles[i][0] for i in range(len(tiles))]
tileCoords = [tiles[i][1].geometry().getInfo()['coordinates'] for i in range(len(tiles))]

In [ ]:
tasks = {}
taskStatus = {}

sd = 1990
ed = 2017

for year in range(sd, ed):
    y = str(year)
    start_d = config.params['years'][y]+config.params['period']['dry']['start']
    end_d = config.params['years'][y]+config.params['period']['dry']['end']
    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(year < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(year in (2002,2011,2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(year > 2002 and year < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)

    filt = filtered.median()

    #Landsat Tiles
    lsClip = [geepy.image.clipImageCollection(filt, tiles[i][1]) for i in range(len(tiles))]

    lsClip = [geepy.image.calcNDVI(lsClip[i]) for i in range(len(lsClip))]

    print('Processing: ', y)
    tasks[str(y)] = [geepy.image.send2drive(lsClip[i].select(['ndvi']), tileCoords[i], 'landsat_ndvi'+str(year)+'_'+tileNames[i], 'LS578AIBA_ndvi') for i in range(len(tiles))]

print('Finish')

In [ ]:
for i in tasks.keys():
    print('Exporting to drive: %s' %(i))
    [tasks[i][j].start() for j in range(len(tasks[i]))]

    taskPoolStatus = False

    while taskPoolStatus == False:
        taskStatus[i] = [tasks[i][k].status()['state'] for k in range(len(tasks[i]))]
        taskPoolStatus = all(status == 'COMPLETED' for status in taskStatus[i])

        if taskPoolStatus:
            print('Exporting data for year %s is completed.' %(i))
            print('Waiting 1 minute.')
            time.sleep(60)